<a href="https://colab.research.google.com/github/MichaelHMorei/arquivos_de_apoio/blob/main/Igtipy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import numpy as np
import os
import zipfile
import requests
from io import BytesIO
#import pyodbc

In [18]:
#Site contendo os dados
url = "https://download.inep.gov.br/microdados/microdados_enem_2020.zip"

In [13]:
#Criando pasta para armazenar os dados
os.makedirs('./enadade2020',exist_ok = True)

In [ ]:
requests.get(url).content)

In [19]:
#Fazendo o download do conteudo
file =  BytesIO(
     requests.get(url).content)
 
# Extraindo o conteudo do arquivo zip na pasta criada.
zip = zipfile.ZipFile(file)
zip.extractall("./enadade2020")

In [29]:
df = pd.read_csv("./enadade2020/DADOS/MICRODADOS_ENEM_2020.csv")

UnicodeDecodeError: ignored